# COMET
Commonsense Transformers for Automatic Knowledge Graph Construction 
- [ACL 2019 Paper](https://arxiv.org/abs/1906.05317)


In [ ]:
# references = {Bosselut2019COMETCT,
#   title={COMET: Commonsense Transformers for Automatic Knowledge Graph Construction},
#   author={Antoine Bosselut and Hannah Rashkin and Maarten Sap and Chaitanya Malaviya and Asli Çelikyilmaz and Yejin Choi},
#   booktitle={Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics (ACL)},
#   year={2019}}
import os 
import urllib 
!git clone https://github.com/atcbosselut/comet-commonsense.git

fatal: destination path 'comet-commonsense' already exists and is not an empty directory.


# 0.Data Load
- Data: [WN18RR](https://github.com/JD-AI-Research-Silicon-Valley/SACN)
    - contains 93,003 triples with 40,943 entities and 11 relation types
    - link prediction dataset created from WN18, a subset of WordNet
        - limit of WN18: test leakage 
            - many text triples are obtained by inverting triples from the training set 
        - WN18RR: created to ensure that the evaluation dataset does not have inverse relation test leakage 
- Data Implementation References: 
    - [WN18RR impelementation](https://github.com/villmow/datasets_knowledge_embedding/tree/master/WN18RR/text)
    - [create_WN18RR](https://github.com/JD-AI-Research-Silicon-Valley/SACN/tree/master/data)
    - [WN18](https://github.com/uclnlp/inferbeddings)

In [ ]:
#!/usr/bin/env python3

# the paths do not exist (we do not want to encourage the usage of WN18) and the filenames are off, but if you put in the WN18 files
# and adjust the path you will generate WN18RR.

predicates_to_remove = [
    '_member_of_domain_topic',
    '_synset_domain_usage_of',
    '_instance_hyponym',
    '_hyponym',
    '_member_holonym',
    '_synset_domain_region_of',  
    '_part_of'
]   

def read_triples(path):     
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split('\t')
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples

def write_triples(triples, path):
    with open(path, 'wt') as f:
        for (s, p, o) in triples:
            f.write('{}\t{}\t{}\n'.format(s, p, o))

train_triples = read_triples('original/wordnet-mlj12-train.txt') 
valid_triples = read_triples('original/wordnet-mlj12-valid.txt')
test_triples = read_triples('original/wordnet-mlj12-test.txt')   

filtered_train_triples = [(s, p, o) for (s, p, o) in train_triples if p not in predicates_to_remove]
filtered_valid_triples = [(s, p, o) for (s, p, o) in valid_triples if p not in predicates_to_remove]
filtered_test_triples = [(s, p, o) for (s, p, o) in test_triples if p not in predicates_to_remove]

write_triples(filtered_train_triples, 'wn18-train.tsv')
write_triples(filtered_valid_triples, 'wn18-valid.tsv')
write_triples(filtered_test_triples, 'wn18-test.tsv')     

# 1.Model Architecture
1. Problem: Difference between conventional KBs and commonsense knowledge 
    - traditional KB: fixed structured format (two entities with a known relation) 
    - commonsense knowledge: open-text description of knowledge (loosely structured)

2. COMET: automatically generate Commonsense Knowledge base graph model 
    - Experiment: ATOMIC, ConcepNet 
    - *BUT here in this implementation code, Experiment: WN18RR*

3. Model Architecture: 
    - basic structure is similar with GPT (Radford et al. 2018) 
    - composed of Transformer Blocks, which contains Multi-head Attention block and Feed-Forward Network 
4. Pre-train Transformer: a seed set of knowledge 
    - during pre-training: 
        - representation revision for knowledge generation 
    - result: generating high-level new tuples (=commonsense knowledge) 


# 2.Training
- import src.data.config as cfg
- import src.data.data as data
- import src.train.utils as train_utils
- import src.train.batch as batch

- import src.evaluate.evaluate as evaluate
- import src.evaluate.generate as gen
- import src.evaluate.sampler as sampling

- import utils.utils as utils

## 2.1 train.py
[src/train/train.py](https://github.com/atcbosselut/comet-commonsense/blob/master/src/train/train.py)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

# !pip install tensorboard
import tensorboard
import torchvision   
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms       

In [ ]:
class Trainer(object):
    def __init__(self, opt, meta, data_loader, model, optimizer):
        self.optimizer = optimizer 
        self.model = model 

        if opt.trainer == 'epoch':
            self.epochs = meta.epochs 
        self.data_loader = data_loader
        self.opt = opt 

        self.losses = {'dev':{}, 'test':{}, 'train':{}}
        self.top_score = None
        self.lrs = {}

        self.batch_variables = {
            'data': self.data_loader,
            'model': self.model,
            'split': 'train'}

        self.do_gen = cfg.do_gen
        self.samplers = {}

    def decide_to_save(self):
        to_save = cfg.save and not cfg.toy 
        to_save = to_save or cfg.test_save 
        print(cfg.save_strategy)
        if cfg.save_strategy == 'best':
            if self.top_score[0] != self.opt.tarin.dynamic.epoch:
                print('DOING IT RIGHT')
                to_save = False 
        return to_save 
    
    def save_model(self, tracked_score):
        lrs = {}
        for i, param_group in enumerate(self.optimizer.param_groups):
            lrs[i] = param_group['lr']
        self.lrs[self.opt.tarin.dynamic.epoch] = lrs 
        to_save = self.decide_to_save()

        if to_save:
            data.save_step(
                self.model, self.data_loader.vocab_encoder,
                self.optimizer, self.opt, 
                self.opt.train.dynamic.epoch, self.lrs)
            
    def log_losses(self, opt, losses):  
        if (not cfg.toy and cfg.save) or cfg.test_save: 
            data.save_eval_file(opt, losses['train'], 'losses', split='train')
            data.save_eval_file(opt, losses['dev'], 'losses',split='dev')   

    def set_logger(self): 
        if cfg.toy: 
            self.logger = SummaryWriter(utils.make_name(
                self.opt, prefix='garbage/logs/', eval_=True, do_epoch=False))
        else: 
            self.logger = SummaryWriter(utils.make_name(
                self.opt, prefix='logs/', eval_=True, do_epoch=False))
        print(f'Logging Tensorboard Files at: {self.logger.logdir}')

    def stop_logger(self):
        self.logger.close()

    def run(self): 
        self.set_logger()
        self.count = 0
        for epoch in range(self.epochs): 
            self.model.train()
            self.opt.train.dynamic.epoch += 1
            self.epoch()
        self.stop_logger()

    def epoch(self): 
        nums = self.reset_losses()

        # initialize progress bar 
        bar = utils.initialize_progress_bar(
            self.data_loader.sequences['train'])
        reset = False 
        
        while not reset: 
            loss, nums, reset = self.do_forward_pass(nums)
            self.do_backward_pass(loss)
            self.update_parameters()

            bar.update(self.opt.train.dynamic.bs)
            self.count += 1

            for loss_name in self.losses['train']:
                self.logger.add_scalar(
                    f'train/{loss_name}',  
                    loss.item() / self.opt.tarin.dynamic.bs,
                    self.count)
                
            if cfg.toy and self.counter(nums) > 300:
                break

            with torch.no_grad():
                self.run_evaluation_cycle()


            self.log_losses(self.opt, self.losses)
            self.update_top_score(self.opt)
            self.save_model(self.get_tracked_score())
            self.data_loader.reset_offsets('train')

    def run_evaluation_cycle(self):
        for split in ['dev','test']:
            self.evaluator.validate(
                self.opt.train.dynamic.epoch, 
                split,
                self.losses[split])
            
            if self.do_gen: 
                gen.do_gen_run(
                    self.opt, self.generator, self.opt.train.dynamic.epoch,
                    split,
                    self.losses[split])
            iter_num = self.opt.train.dynamic.epoch 

            for loss_name in self.losses[split]:
                self.logger.add_scalar(
                    f'{split}/{loss_name}',
                    self.losses[split][loss_name][iter_num],
                    iter_num)
                
    def clip_gradients(self):
        if self.opt.train.static.clip:
            torch.nn.utils.clip_grad_norm(
                self.model.parameters(), self.opt.train.static.clip)
            
    def do_forward_pass(self, nums): 
        token_loss, nums, reset = self.batch(
            self.opt, nums, self.losses['train'],
            self.batch_variables)
        return token_loss, nums, reset 

    def do_backward_pass(self, loss): 
        loss.backward()

    def update_parameters(self):   
        if self.opt.model == 'lstm': 
                self.clip_graients()
        self.optimizer.step()
        self.optimizer.zero_grad()
    
    def reset_losses(self): 
        loss_names = set([i.rstrip('maicro').rstrip('_') for 
                            i in self.losses['train'].keys()])
        return self.initialize_losses(list(loss_names))   
